In [171]:
import numpy as np

In [172]:
fashion = np.genfromtxt('data/aij_fashion.csv', delimiter=",",dtype=int)

In [173]:
def format_data(aij_fashion):
    data = {}
    for (worker,influencer,label) in aij_fashion:
        if influencer not in data:
            data[influencer] = {}
        result_label = [0]
        if label == 1:
             result_label = [1]
        data[influencer][worker] = result_label
    return data

In [174]:
fashion = format_data(fashion)

In [175]:
def fashion_to_counts(fashions):
    """
    Convert a matrix of fashions to count data
    Args:
        fashions: dictionary of label {influencers:{workers:[labels]}}
    Returns:
        influencers: list of influencers
        workers: list of workers
        choices: list of choices (1 or 0 in our case)
        counts: 3d array of counts: [influencers x workers]
    """
    influencers = list(fashions)
    nInfluencers = len(influencers)

    workers = set()
    choices = set()
    for i in influencers:
        i_workers = list(fashions[i])
        for k in i_workers:
            if k not in workers:
                workers.add(k)
            ik_label = fashions[i][k]
            choices.update(ik_label)

    choices = list(choices)
    choices.sort()
    nChoices = len(choices)
    
    workers = list(workers)
    nWorkers = len(workers)

    # create a 3d array to hold counts
    counts = np.zeros([nInfluencers, nWorkers, nChoices])

    # convert responses to counts
    for influencer in influencers:
        i = influencers.index(influencer)
        for worker in fashions[influencer].keys():
            k = workers.index(worker)
            for label in fashions[influencer][worker]:
                j = choices.index(label)
                counts[i, k, j] += 1

    return (influencers, workers, choices, counts)

In [176]:
(influencers, workers, choices, counts) = fashion_to_counts(fashion)

In [177]:
def majority_voting(counts):
    [nInfluencers, nWorkers, nChoices] = np.shape(counts)
    responses_sums = np.sum(counts, 1)
    result = np.zeros([nInfluencers, nChoices])
    for p in range(nInfluencers):
        result[p, :] = responses_sums[p, :] / np.sum(responses_sums[p, :], dtype=float)
    return result

In [178]:
influencers_label = majority_voting(counts)

In [179]:
influencers_label

array([[0.98770492, 0.01229508],
       [0.95081967, 0.04918033],
       [0.99180328, 0.00819672],
       ...,
       [0.99590164, 0.00409836],
       [0.99590164, 0.00409836],
       [0.99590164, 0.00409836]])

In [180]:
def m_step(counts, influencers_label):
    [nInfluencers, nWorkers, nChoices] = np.shape(counts)

    # compute class marginals
    class_marginals = np.sum(influencers_label, 0) / float(nInfluencers)

    # compute error rates
    error_rates = np.zeros([nWorkers, nChoices, nChoices])
    for k in range(nWorkers):
        for j in range(nChoices):
            for l in range(nChoices):
                error_rates[k, j, l] = np.dot(
                    influencers_label[:, j], counts[:, k, l])
            sum_over_responses = np.sum(error_rates[k, j, :])
            if sum_over_responses > 0:
                error_rates[k, j, :] = error_rates[
                    k, j, :] / float(sum_over_responses)

    return (class_marginals, error_rates)

In [181]:
(class_marginals, error_rates) = m_step(counts,influencers_label)

In [185]:
def e_step(counts, class_marginals, error_rates):
    [nInfluencers, nWorkers, nChoices] = np.shape(counts)

    influencers_label = np.zeros([nInfluencers, nChoices])

    for i in range(nInfluencers):
        for j in range(nChoices):
            estimate = class_marginals[j]
            estimate *= np.prod(np.power(error_rates[:, j, :], counts[i, :, :]))
            influencers_label[i, j] = estimate
        question_sum = np.sum(influencers_label[i, :])
        if question_sum > 0:
            influencers_label[i, :] = influencers_label[i, :] / float(question_sum)
    return influencers_label

In [183]:
influencers_label = e_step(counts, class_marginals, error_rates)

In [184]:
influencers_label

array([[9.96818451e-01, 3.18154879e-03],
       [3.67969483e-07, 9.99999632e-01],
       [9.99680657e-01, 3.19343028e-04],
       ...,
       [9.99272876e-01, 7.27124197e-04],
       [9.99556229e-01, 4.43770599e-04],
       [9.99904300e-01, 9.57003684e-05]])